<a href="https://colab.research.google.com/github/DevEderNO/netflix-classification/blob/master/netflix_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import xlrd
import itertools
import pylab
import matplotlib.pyplot as plt
from datetime import datetime as dt
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression, SelectKBest, SelectPercentile, chi2
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
%matplotlib inline

# Mostrar todas as colunas
pd.pandas.set_option('display.max_columns', None)

# Suprimir as mensagens de alerta
import warnings
warnings.filterwarnings('ignore')

import pymc3 as pm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
ds = pd.read_csv('netflix_titles.csv')
print('Formato de arquivo de treino {}'.format(ds.shape))
ds.head()

Formato de arquivo de treino (6234, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


#**Abaixo estão as descrições das colunas do nosso Dataset**

1. **show_id** - Identificador unico para Filmes / Tv Show
2. **type** - Tipo de A Filmes / Tv Show
3. **title** - Titulo dos Filmes / Tv Show
4. **director** - Diretor de Filmes
5. **cast** - Autores involvidos nos Filmes / Tv Show
6. **country** - Região onde os Filmes / Tv Show foram produzidos
7. **date_added** - Data que foi adicionado na Netflix
8. **release_year** - Ano de lançamento real dos Filmes / Tv Show
9. **rating** - Classificação dos Filmes / Tv Show
10. **duration** - Duração total em minutos ou numeros de estações
11. **listed_in** - Listado nos Generos
12. **description** - Descrição do sumário

In [0]:
# Verificando os tipos de dados contidos no Dataset
ds.dtypes

show_id          int64
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

In [0]:
ds.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
show_id,6234.0,7.670368e+07,1.094296e+07,247747.0,80035801.75,80163367.0,80244888.75,81235729.0
release_year,6234.0,2.013359e+03,8.811620e+00,1925.0,2013.00,2016.0,2018.00,2020.0


# **Abaixo farei uma divisão dos dados em categóric e numérico para ter uma visão mais detalhada sobre as features.**

In [0]:
#Nomes das colunas Categóricas do Dataset
cat_col = [var for var in ds.columns if ds[var].dtype == 'O']
ds.loc[:,cat_col].head()

,type,title,director,cast,country,date_added,rating,duration,listed_in,description
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [0]:
# Nome das colunas numéricas do Dataset
num_col = [var for var in ds.columns if ds[var].dtype != 'O']
ds.loc[:, num_col].head(3)

,show_id,release_year
0,81145628,2019
1,80117401,2016
2,70234439,2013


#**Para as features numéricas temos o seguinte cenário:**

* 1 variável Id: show_id (chave primária)
* 1 variável contínua: release_year.

In [0]:
# Percentual de dados faltantes na base de Treino
round(ds.isnull().mean() *100,2)

show_id          0.00
type             0.00
title            0.00
director        31.58
cast             9.14
country          7.64
date_added       0.18
release_year     0.00
rating           0.16
duration         0.00
listed_in        0.00
description      0.00
dtype: float64

In [0]:
print('Total de FIlmes / TV Show no DataSet de treino {}'.format(ds.shape[0]))
print('Total de FIlmes / TV Show com todos os registros preenchidos (linhas) {}'.format(ds.dropna().shape[0]))
print('Percentual de dados com 100% do preenchimento dos dados {}'.format(round(ds.dropna().shape[0] / len(ds) *100,2)))

Total de FIlmes / TV Show no DataSet de treino 6234
Total de FIlmes / TV Show com todos os registros preenchidos (linhas) 3774
Percentual de dados com 100% do preenchimento dos dados 60.54


#**As colunas director, cast, country, data_added e rating precisa ser tratada para valores faltantes.**

## Vamos começar pela coluna rating que e a coluna que possui menos valores faltantes.

In [0]:
#Verificando a quantidade de registros que não possuem a informação rating
print('Quantidade de linhas sem a informação rating {}'.format(ds[ds.rating.isnull()].shape[0]))
print('Quantidade de linhas sem a informação date_added {}'.format(ds[ds.date_added.isnull()].shape[0]))
print('Quantidade de linhas sem a informação country {}'.format(ds[ds.country.isnull()].shape[0]))
print('Quantidade de linhas sem a informação cast {}'.format(ds[ds.cast.isnull()].shape[0]))

Quantidade de linhas sem a informação rating 10
Quantidade de linhas sem a informação date_added 11
Quantidade de linhas sem a informação country 476
Quantidade de linhas sem a informação cast 570


## Como a poucos registros que não possuem informações irei apenas remover esses registros.

In [0]:
# Removendo registros sem informações por meio de filtro.
ds = ds[ds.cast.notnull() & ds.date_added.notnull() & ds.country.notnull() & ds.rating.notnull() ]
round(ds.isnull().mean() *100,2)

show_id          0.0
type             0.0
title            0.0
director        28.2
cast             0.0
country          0.0
date_added       0.0
release_year     0.0
rating           0.0
duration         0.0
listed_in        0.0
description      0.0
dtype: float64

#**A coluna director precisa ser tratada para valores faltantes.**

A coluna director possui muitos dados faltantes neste caso sera melhor remover a coluna.

In [0]:
ds = ds.drop(['director'], axis=1)
ds.shape[0]

5256

#**A coluna date_added possui informações de mês e ano nesse caso podemos separalos para termos uma maior especificidade dos dados**

## Para isso vamos criar 2 novas colunas **_year_added_** e **_month_added_** e remover a coluna **_date_added_**

In [0]:
ds['year_added'] = ds['date_added'].apply(lambda x : x.split(', ')[-1])
ds['month_added'] = ds['date_added'].apply(lambda x : x.lstrip().split(' ')[0])

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'][::-1]
df = ds.groupby('year_added')['month_added'].value_counts().unstack().fillna(0)[month_order].T
ds.drop(['date_added'], axis=1)

,show_id,type,title,cast,country,release_year,rating,duration,listed_in,description,year_added,month_added
0,81145628,Movie,Norm of the North: King Sized Adventure,"Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,2019,September
1,80117401,Movie,Jandino: Whatever it Takes,Jandino Asporaat,United Kingdom,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,2016,September
2,70234439,TV Show,Transformers Prime,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob...",2018,September
3,80058654,TV Show,Transformers: Robots in Disguise,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,2018,September
4,80125979,Movie,#realityhigh,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,2017,September
...,...,...,...,...,...,...,...,...,...,...,...,...
6218,80162994,TV Show,Talking Tom and Friends,"Colin Hanks, Tom Kenny, James Adomian, Lisa Sc...","Cyprus, Austria, Thailand",2017,TV-G,2 Seasons,"Kids' TV, TV Comedies",Full of funny one-liners and always ready for ...,2019,April
6219,80186475,TV Show,Pokémon the Series,"Sarah Natochenny, Laurie Hymes, Jessica Paquet...",Japan,2019,TV-Y7-FV,2 Seasons,"Anime Series, Kids' TV",Ash and his Pikachu travel to the Alola region...,2019,April
6220,70272742,TV Show,Justin Time,"Gage Munroe, Scott McCord, Jenna Warren",Canada,2012,TV-Y,2 Seasons,Kids' TV,"In Justin's dreams, he and his imaginary frien...",2016,April
6221,80067942,TV Show,Terrace House: Boys & Girls in the City,"You, Reina Triendl, Ryota Yamasato, Yoshimi To...",Japan,2016,TV-14,2 Seasons,"International TV Shows, Reality TV",A new set of six men and women start their liv...,2016,April


#**Trantando os dados que seram usados para o treinar**

In [0]:
ds['country'] = ds.country.astype('category')
ds['country'].cat.categories
ds['country'] = ds['country'].cat.codes

In [0]:
ds['rating'] = ds.rating.astype('category')
ds['rating'].cat.categories
ds['rating'] = ds['rating'].cat.codes

#**Iniciando o treino dos dados**

## O teste utilizado e para tentar classificar o __Tipo__ de um novo registro usando a região como parâmetro.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(ds[['country']], ds['type'], random_state=0)

In [0]:
print("X_train shape: {}".format(X_train.shape)) #
print("y_train shape: {}".format(y_train.shape)) #
print("X_test shape: {}".format(X_test.shape)) #
print("y_test shape: {}".format(y_test.shape)) #

X_train shape: (3942, 1)
y_train shape: (3942,)
X_test shape: (1314, 1)
y_test shape: (1314,)


In [0]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [0]:
print("Test set predictions:\n {}".format(y_pred))

Test set predictions:
 ['TV Show' 'TV Show' 'TV Show' ... 'Movie' 'TV Show' 'TV Show']


#**A acuracia do teste está muito baixa o que indica que temos que melhorar o nosso dataset**

In [0]:
print("Test set score: {:.2f}".format(np.mean(y_pred == y_test)))

Test set score: 0.47
